In [34]:
from transformers import ASTModel, ASTConfig
import torch
import torch.nn as nn
from lumen_irmas.modeling.models import StudentAST, ASTPretrained
import torch.nn.functional as F
import numpy as np

In [7]:
teacher = ASTPretrained(11)
student = StudentAST(11)

Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.dense.bias', 'classifier.layernorm.weight', 'classifier.layernorm.bias', 'classifier.dense.weight']
- This IS expected if you are initializing ASTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ASTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
[p.shape for p in student.base_model.embeddings.parameters()]

[torch.Size([1, 1, 384]),
 torch.Size([1, 1, 384]),
 torch.Size([1, 1214, 384]),
 torch.Size([384, 1, 16, 16]),
 torch.Size([384])]

In [28]:
[tuple(p.shape) for p in teacher.base_model.embeddings.parameters()]

[(1, 1, 768), (1, 1, 768), (1, 1214, 768), (768, 1, 16, 16), (768,)]

In [55]:
torch.rand(1,1,128).squeeze().shape

torch.Size([128])

In [195]:
model1_shapes = [np.array(param.shape) for param in teacher.base_model.parameters()]
model2_shapes = [np.array(param.shape) for param in student.base_model.parameters()]

In [196]:
model1_shapes

[array([  1,   1, 768]),
 array([  1,   1, 768]),
 array([   1, 1214,  768]),
 array([768,   1,  16,  16]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([3072,  768]),
 array([3072]),
 array([ 768, 3072]),
 array([768]),
 array([768]),
 array([768]),
 array([768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([3072,  768]),
 array([3072]),
 array([ 768, 3072]),
 array([768]),
 array([768]),
 array([768]),
 array([768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([3072,  768]),
 array([3072]),
 array([ 768, 3072]),
 array([768]),
 array([768]),
 array([768]),
 array([768]),
 array([768]),
 array([768, 768]),
 array([768]),
 array([768, 768]

In [268]:
new_params = {}

# Iterate over the parameters in the first model
for i, (name, param) in enumerate(teacher.base_model.named_parameters()):
    # Scale the parameter using interpolate if its shape is different from that of the second model
    target_param = student.base_model.state_dict()[name]
    if param.shape != target_param.shape:
        
        squeeze_count = 0
        permuted = False
        while param.ndim < 4:
            param = param.unsqueeze(0)
            squeeze_count += 1
        
        if param.shape[0] > 1:
            param = param.permute(1,2,3,0)
            target_param = target_param.permute(1,2,3,0)
            permuted = True
            
        if target_param.ndim < 2:
            target_param = target_param.unsqueeze(0)
        
        scaled_param = torch.nn.functional.interpolate(param, size=(target_param.shape[-2:]), mode="bilinear")
        
        while squeeze_count > 0:
            scaled_param = scaled_param.squeeze(0)
            squeeze_count -= 1
        
        if permuted:
            scaled_param = scaled_param.permute(-1,0,1,2)
        
    else:
        scaled_param = param
    new_params[name] = scaled_param


In [265]:
student.base_model.load_state_dict(new_params)

<All keys matched successfully>

In [215]:
param.shape

torch.Size([768, 1, 16, 16])

In [186]:
scaled_param.shape

torch.Size([1, 1, 1, 384])

In [85]:
student.base_model.state_dict()[name].shape

torch.Size([1, 1, 384])

In [223]:
F.interpolate(torch.rand(1,16,16,768), size=(16,384), mode="bilinear").shape

torch.Size([1, 16, 16, 384])

In [160]:
param.shape

torch.Size([1, 1, 768])

In [1]:
from lumen_irmas.modeling.models import Ensemble, StudentAST

In [2]:
ensemble = Ensemble([StudentAST(11), StudentAST(11)])

In [5]:
ensemble = ensemble.to("meta")